In [1]:
from selenium import webdriver
from time import sleep
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException

driver = webdriver.Chrome()
url = 'https://cq.esf.fang.com/house-a060-b04863/'
driver.get(url)

data = []

for page_num in range(1, 21):
    print(f"正在抓取第 {page_num} 页数据...")
    sleep(3)
    
    try:
        rows = driver.find_elements(By.CSS_SELECTOR, 'dl')
        print(f"本页找到 {len(rows)} 个房源")
        
        for i, row in enumerate(rows):
            # 提取总价格 - 精准定位到价格区域的span
            try:
                # 定位到价格区域的span元素
                price_element = row.find_element(By.CSS_SELECTOR, 'dd.price_right span.red b')
                price_text = price_element.text
                total_price = str(int(float(price_text) * 10000))  # 先用float计算，再转int去掉小数
            except NoSuchElementException:
                total_price = "总价未知"
            except ValueError:
                total_price = "总价未知"
            
            # 提取平米价格 - 不携带单位
            try:
                price_per_element = row.find_element(By.CSS_SELECTOR, 'dd.price_right span:not(.red)')
                price_per_text = price_per_element.text
                price_per_match = re.search(r'(\d+)\s*元/㎡', price_per_text)
                if price_per_match:
                    price_per_m2 = price_per_match.group(1)  # 只保留数字
                else:
                    price_per_m2 = "平米价未知"
            except NoSuchElementException:
                price_per_m2 = "平米价未知"
            
            # 提取面积 - 精准定位到 tel_shop 类中的面积
            try:
                # 精准定位到 p class="tel_shop" 元素
                tel_shop_element = row.find_element(By.CSS_SELECTOR, 'p.tel_shop')
                tel_shop_text = tel_shop_element.text
                
                # 在 tel_shop 文本中查找面积
                area_match = re.search(r'(\d+\.?\d*)\s*㎡', tel_shop_text)
                if area_match:
                    area = area_match.group(1)  # 只保留数字
                else:
                    area = "面积未知"
            except NoSuchElementException:
                area = "面积未知"
            
            data.append([total_price, price_per_m2, area])
            print(f"  房源 {i+1}: 总价={total_price}, 平米价={price_per_m2}, 面积={area}")
            
    except Exception as e:
        print(f"第 {page_num} 页数据提取失败: {e}")
        continue
    
    # 翻页
    if page_num < 20:
        try:
            next_page = driver.find_element(By.XPATH, '//a[contains(text(), "下一页")]')
            next_page.click()
            print("成功点击下一页")
        except NoSuchElementException:
            try:
                next_page = driver.find_element(By.CSS_SELECTOR, '.next')
                next_page.click()
                print("成功点击下一页(备用)")
            except NoSuchElementException:
                print("没有找到下一页按钮，提前结束抓取")
                break

df = pd.DataFrame(data, columns=['总价格', '平米价格', '面积'])
print(f"总共抓取了 {len(df)} 条数据")

# 直接保存原始数据，不进行清洗
print("\n前10条数据预览:")
print(df.head(10))

# 保存数据
df.to_csv('房产数据.csv', index=False, encoding='utf-8-sig')
print("数据已保存到 房产数据.csv")

driver.quit()

正在抓取第 1 页数据...
本页找到 62 个房源
  房源 1: 总价=1190000, 平米价=13612, 面积=87.42
  房源 2: 总价=520000, 平米价=6730, 面积=77.26
  房源 3: 总价=1200000, 平米价=9302, 面积=129
  房源 4: 总价=750000, 平米价=7916, 面积=94.74
  房源 5: 总价=1650000, 平米价=13200, 面积=125
  房源 6: 总价=1660000, 平米价=5443, 面积=304.94
  房源 7: 总价=1320000, 平米价=13678, 面积=96.5
  房源 8: 总价=1160000, 平米价=11485, 面积=101
  房源 9: 总价=870000, 平米价=7591, 面积=114.6
  房源 10: 总价=830000, 平米价=8859, 面积=93.69
  房源 11: 总价=1400000, 平米价=9459, 面积=148
  房源 12: 总价=1400000, 平米价=9459, 面积=148
  房源 13: 总价=5000000, 平米价=15990, 面积=312.69
  房源 14: 总价=3850000, 平米价=11809, 面积=326
  房源 15: 总价=630000, 平米价=7078, 面积=89
  房源 16: 总价=2550000, 平米价=13350, 面积=191
  房源 17: 总价=1160000, 平米价=9382, 面积=123.64
  房源 18: 总价=1160000, 平米价=11485, 面积=101
  房源 19: 总价=650000, 平米价=7386, 面积=88
  房源 20: 总价=1390000, 平米价=11487, 面积=121
  房源 21: 总价=930000, 平米价=7622, 面积=122
  房源 22: 总价=1490000, 平米价=10719, 面积=139
  房源 23: 总价=830000, 平米价=8736, 面积=95
  房源 24: 总价=930000, 平米价=7685, 面积=121
  房源 25: 总价=960000, 平米价=7111, 面积=135
  房源 26: 总价=850